import all the required modules.

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [2]:
train = pd.read_csv("data/Opportunities_train.csv")    # Read the data
train.head()

,Opp No,Opp Type,Status,Region,Country,BU,Sector,Customer Group,Customer,Product,...,Cust Dec Date,Go Live Date,Date Exp Gain,Payment Terms,Contract Term,Milestone,Prob to win %,BCA_Status,Ann Rev,Ann GP
0,199060,1. New,4. Early Lead,Latin America,Brazil,CLA,Consumer,UNILEVER,Unilever,NaN,...,NaN,NaN,22/09/2017,NaN,< 1 Year,Early Lead,NaN,NaN,NaN,NaN
1,224663,1. New,1. Gain,Latin America,Brazil,CLA,Consumer,PROCTER & GAMBLE,Procter & Gamble,Warehousing,...,09/12/2015,03/10/2016,24/03/2016,35,1-2 Years,Customer commitment_ Move to gain,100,Approved,79.68,86.54
2,224709,2. Renewal,1. Gain,North America,USA,CLA,Consumer,PROCTER & GAMBLE,Procter & Gamble,Warehousing,...,14/12/2015,01/11/2016,05/04/2016,15,2-3 Years,Customer commitment_ Move to gain,100,Approved,60.65,31.23
3,225808,2. Renewal,1. Gain,North America,USA,CLA,Consumer,PROCTER & GAMBLE,Procter & Gamble,Warehousing,...,01/05/2015,01/07/2015,17/06/2016,75,2-3 Years,Customer commitment_ Move to gain,100,Approved,19.44,11.67
4,226704,2. Renewal,1. Gain,Latin America,Brazil,CLA,Consumer,UNILEVER,Unilever,Warehousing,...,29/02/2016,01/01/2016,12/04/2016,90,< 1 Year,Customer commitment_ Move to gain,100,Approved,211.71,16.93


In [3]:
# Impute median Payment Terms for NA Payment Terms values
new_payment_terms_var = np.where(train["Payment Terms"].isnull(), # Logical check
                       60,                       # Value if check is true
                       train["Payment Terms"])     # Value if check is false

train["Payment Terms"] = new_payment_terms_var 

# Impute median  Prob to win %  for NA Prob to win %  values
new_prob_to_win_var = np.where(train["Prob to win %"].isnull(), # Logical check
                       70,                       # Value if check is true
                       train["Prob to win %"])     # Value if check is false

train["Prob to win %"] = new_payment_terms_var 

# Impute median Ann Rev for NA Ann Rev values
new_ann_rev_var = np.where(train["Ann Rev"].isnull(), # Logical check
                       73,                       # Value if check is true
                       train["Ann Rev"])     # Value if check is false

train["Ann Rev"] = new_ann_rev_var 

# Impute median Ann GP for NA Ann GP values
new_ann_gp_var = np.where(train["Ann GP"].isnull(), # Logical check
                       28,                       # Value if check is true
                       train["Ann GP"])     # Value if check is false

train["Ann GP"] = new_ann_gp_var 

In [4]:
# Set the seed
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

# Convert some variables to numeric
train["Opp Type"] = label_encoder.fit_transform(train["Opp Type"])
train["Region"] = label_encoder.fit_transform(train["Region"])
train["Country"] = label_encoder.fit_transform(train["Country"])
train["BU"] = label_encoder.fit_transform(train["BU"])
train["Sector"] = label_encoder.fit_transform(train["Sector"])
train["Customer Group"] = label_encoder.fit_transform(train["Customer Group"])
train["Customer"] = label_encoder.fit_transform(train["Customer"])
train["Product"] = label_encoder.fit_transform(train["Product"])
train["Contract Term"] = label_encoder.fit_transform(train["Contract Term"])
train["Milestone"] = label_encoder.fit_transform(train["Milestone"])
train["BCA_Status"] = label_encoder.fit_transform(train["BCA_Status"])

# Initialize the model
rf_model = RandomForestClassifier(n_estimators=1000, # Number of trees
                                  max_features=2,    # Num features considered
                                  oob_score=True)    # Use OOB scoring*

features = ["Opp Type","Region","Country","BU","Sector","Customer Group","Customer","Product","Payment Terms","Contract Term","Milestone","Prob to win %","BCA_Status","Ann Rev","Ann GP"]

# Train the model
rf_model.fit(X=train[features],
             y=train["Status"])

print("OOB accuracy: ")
print(rf_model.oob_score_)

OOB accuracy: 
0.822222222222


Since random forest models involve building trees from random subsets or "bags" of data, model performance can be estimated by making predictions on the out-of-bag (OOB) samples instead of using cross validation. You can use cross validation on random forests, but OOB validation already provides a good estimate of performance and building several random forest models to conduct K-fold cross validation with random forest models can be computationally expensive.
The random forest classifier assigns an importance value to each feature used in training. Features with higher importance were more influential in creating the model, indicating a stronger association with the response variable. Let's check the feature importance for our random forest model:

In [5]:
for feature, imp in zip(features, rf_model.feature_importances_):
    print(feature, imp)

('Opp Type', 0.034856833290826117)
('Region', 0.034898759290593988)
('Country', 0.032323574266527363)
('BU', 0.051020852670909264)
('Sector', 0.026028107088887351)
('Customer Group', 0.050392707954248092)
('Customer', 0.055119855204460166)
('Product', 0.11463584957639236)
('Payment Terms', 0.035253952105767691)
('Contract Term', 0.042095254279358452)
('Milestone', 0.24270100662193567)
('Prob to win %', 0.033796720388494059)
('BCA_Status', 0.11175710229570097)
('Ann Rev', 0.060184429388258978)
('Ann GP', 0.074934995577639135)


Feature importance can help identify useful features and eliminate features that don't contribute much to the model.
As a final exercise, let's use the random forest model to make predictions on the opportunity test set to see how our actual generalization performance compares to the OOB estimate:

In [6]:
# Read and prepare test data
test = pd.read_csv("data/Opportunities_test.csv")    # Read the data

# Impute median Payment Terms for NA Payment Terms values
new_payment_terms_var = np.where(test["Payment Terms"].isnull(), # Logical check
                       60,                       # Value if check is true
                       test["Payment Terms"])     # Value if check is false

test["Payment Terms"] = new_payment_terms_var 

# Impute median  Prob to win %  for NA Prob to win %  values
new_prob_to_win_var = np.where(test["Prob to win %"].isnull(), # Logical check
                       70,                       # Value if check is true
                       test["Prob to win %"])     # Value if check is false

test["Prob to win %"] = new_payment_terms_var 

# Impute median Ann Rev for NA Ann Rev values
new_ann_rev_var = np.where(test["Ann Rev"].isnull(), # Logical check
                       73,                       # Value if check is true
                       test["Ann Rev"])     # Value if check is false

test["Ann Rev"] = new_ann_rev_var 

# Impute median Ann GP for NA Ann GP values
new_ann_gp_var = np.where(test["Ann GP"].isnull(), # Logical check
                       28,                       # Value if check is true
                       test["Ann GP"])     # Value if check is false

test["Ann GP"] = new_ann_gp_var 

# Convert some variables to numeric
test["Opp Type"] = label_encoder.fit_transform(test["Opp Type"])
test["Region"] = label_encoder.fit_transform(test["Region"])
test["Country"] = label_encoder.fit_transform(test["Country"])
test["BU"] = label_encoder.fit_transform(test["BU"])
test["Sector"] = label_encoder.fit_transform(test["Sector"])
test["Customer Group"] = label_encoder.fit_transform(test["Customer Group"])
test["Customer"] = label_encoder.fit_transform(test["Customer"])
test["Product"] = label_encoder.fit_transform(test["Product"])
test["Contract Term"] = label_encoder.fit_transform(test["Contract Term"])
test["Milestone"] = label_encoder.fit_transform(test["Milestone"])
test["BCA_Status"] = label_encoder.fit_transform(test["BCA_Status"])

In [7]:
# Make test set predictions
test_preds = rf_model.predict(X= test[features])

submission = pd.DataFrame({"Opp No":test["Opp No"],
                           "Status":test_preds})

# Save submission to CSV
submission.to_csv("data/result.csv", 
                  index=False)        # Do not save index values